In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../../notes and data/titanic/train.csv')
df = pd.read_excel('../../notes and data/titanic/titanic3.xlsx')
test = pd.read_csv('../../notes and data/titanic/test.csv')
test = test.drop(columns=['Unnamed: 12','Name','Ticket'])
train = train.drop(columns=['Name','Ticket'])

In [3]:
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Age'] = train['Age'].fillna(train['Age'].median())
train =  train.drop(columns=['Cabin'])
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
print(train.isnull().sum()[train.isnull().sum() > 0].to_string())

Series([], )


In [4]:
train = pd.get_dummies(train, drop_first=True).astype(int)

In [5]:
x_train = train.drop(columns=['Survived'])
y_train = train['Survived']
x_train = np.array(x_train)
y_train = np.array(y_train)

In [6]:
# test = pd.get_dummies(test, drop_first=True).astype(int)
test
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test =  test.drop(columns=['Cabin'])
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode()[0])
print(test.isnull().sum()[test.isnull().sum() > 0].to_string())
test = pd.get_dummies(test, drop_first=True).astype(int)
test
x_test = test.drop(columns=['Survived'])
y_test = test['Survived']
x_test = np.array(x_test)
y_test = np.array(y_test)

Series([], )


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train,y_train)
b = lr_model.intercept_
w = lr_model.coef_
print(b,w)

[5.00439733] [[ 8.60209454e-05 -1.06476403e+00 -3.82059781e-02 -3.13591391e-01
  -8.14583824e-02  2.20098100e-03 -2.62141946e+00 -2.81640873e-02
  -3.83405552e-01]]


In [8]:
train_score = lr_model.score(x_train, y_train)
test_score = lr_model.score(x_test,y_test)
print(train_score)
print(test_score)

0.7991021324354658
0.937799043062201


In [9]:
import copy
import numpy as np
import math

# sigmoid function
def sigmoid(z):
    z = np.clip(z, -500, 500)  # Clip z to prevent overflow
    return 1 / (1 + np.exp(-z))


# Cost function
def costFn(x, y, w, b):
    m = x.shape[0]
    z = np.dot(x, w) + b
    f = sigmoid(z)
    f = np.clip(f, 1e-10, 1 - 1e-10)  # Clip values to avoid log(0)
    cost = -np.mean(y * np.log(f) + (1 - y) * np.log(1 - f))
    return cost

# Gradient function
def gradientFn(x, y, w, b):
    m, n = x.shape
    djw = np.zeros((n,))
    djb = 0.0
    for i in range(m):
        z = np.dot(w, x[i]) + b
        f = sigmoid(z)
        err = f - y[i]
        for j in range(n):
            djw[j] += err * x[i, j]
        djb += err
    djw /= m
    djb /= m
    return djw, djb

# Gradient descent function
def gradientDecent(x, y, w_in, b_in, alpha, iters):
    w = copy.deepcopy(w_in)  # avoid modifying global w within function
    b = b_in
    for i in range(iters):
        djw, djb = gradientFn(x, y, w, b)
        w = w- alpha * djw
        b -= alpha * djb
        if i % math.ceil(iters / 10) == 0:
            cost = costFn(x, y, w, b)
            print(f"Iteration {i:4d}: Cost {cost:.4f}")
    return w, b


In [10]:
def predict(x, w, b, threshold):
    m = x.shape[0]
    predictions = np.zeros(m)
    for i in range(m):
        z = np.dot(w, x[i]) + b
        prob = sigmoid(z)
        # Classify as 1 if probability is above the threshold
        predictions[i] = 1 if prob >= threshold else 0
    return predictions

In [54]:
# Initial parameters
initial_w = np.zeros_like(x_train[0])  # Shape matches the number of features
initial_b = 0.
iterations = 1000
# alpha = 1.0e-5
alpha = 0.01
# Run gradient descent
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)  # Don't fit on the test set, only transform

# Now use x_train_scaled and x_test_scaled in your gradient descent function
w, b = gradientDecent(x_train_scaled, y_train, initial_w, initial_b, alpha, iterations)
# w, b = gradientDecent(x_train, y_train, initial_w, initial_b, alpha, iterations)
print("Final weights:", w)
print("Final bias:", b)

Iteration    0: Cost 0.6918
Iteration  100: Cost 0.5954
Iteration  200: Cost 0.5439
Iteration  300: Cost 0.5135
Iteration  400: Cost 0.4943
Iteration  500: Cost 0.4814
Iteration  600: Cost 0.4723
Iteration  700: Cost 0.4658
Iteration  800: Cost 0.4609
Iteration  900: Cost 0.4571
Final weights: [ 0.00775397 -0.55660897 -0.25559526 -0.19473987 -0.01937382  0.24072224
 -0.98992766 -0.01554222 -0.173857  ]
Final bias: -0.4721488762658939


In [55]:
threshold = 0.5
y_pred = predict(x_test_scaled, w, b,threshold)
y_pred_train = predict(x_train_scaled, w, b,threshold)

In [56]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(accuracy,accuracy_train)

0.9449760765550239 0.7946127946127947
